# Latent Dirichlet Allocation

**def** dirichlet distribution

$\mu = \begin{bmatrix} \mu_1 \\ \vdots \\ \mu_k \end{bmatrix}$  
$\sum_i \mu_i = 1$  
$\alpha \in \mathbb{R}^k$, $\alpha_0 = \sum_i \alpha_i$  
then $\mu \sim Dirichlet(\alpha)$ iff $ p(\mu) = \frac{\Gamma(\alpha_0)}{\prod \gamma(\alpha_i)} \prod \mu_i^{\alpha_i - 1}$


**def** discrete distribution

$P(x \mid \mu) = \prod_i^k \mu_i^{x_i}$

**def** LDA generative model

for each $d$, draw $\theta_{k | d} \sim Dirichlet(\theta_d \mid \alpha)$  
for each $k$, draw $\phi_{i | k} \sim Dirichlet(\phi_k \mid \beta)$  
for each $d$, for each location $j$ in $d$, draw

## Variational inference for LDA

**notation**

* $d$ : document
* $i$ : location index
* $v$ : index to vocabulary (words)
* $k$ : topic
* $\theta_d$ : topic distribution of document $d$
    * $\theta_{k | d}$
* $\phi_k$ : word distribution of topic $k$
    * $\phi_{v | k}$
* $z_{d_i}$ : hidden topic of document $d$ at location $d_i$
    * $z_{d_i, k} = 1 \iff$ location $i$ in document $d$ has topic $k$
* $w_{d_i}$ : word at location $d, i$
    * $w_{d_i, v} = 1 \iff w_{d_i} = v$

$L = (\prod_d p(\theta_d | \alpha)) \times (\prod_k p(\phi_k | \beta)) \times (\prod_d \prod_i p(z_{d_i} | \theta_d)) \times (\prod_d \prod_i p(w_{d_i} | z_{d_i}, \{\phi\}))$  
dirichlet prior for $\theta$ times dirichlet prior for $\phi$ times discrete distribution for $z$ times discrete distribution for $w$

$= (\prod_d \frac{\Gamma(\alpha_0)}{\prod_k \Gamma(\alpha_k)} \prod_k \theta_{k|d}^{\alpha_k - 1}) \times
(\prod_k \frac{\Gamma(\beta_0)}{\prod_v \Gamma(\beta_v)} \prod_v \phi_{v|k}^{\beta_v - 1}) \times
(\prod_d \prod_i \prod_k \theta_{k|d}^{z_{d_i, k}}) \times
(\prod_d \prod_i \prod_k (\prod_v \phi_{v|k}^{w_{d_i, v}})^{z_{d_i}, k})$

$\log L = (\sum_d(\log \Gamma(\alpha_0) - \sum_k \log \Gamma(\alpha_k) + \sum_k \alpha_k - 1) \log \theta_{k|d})) + 
(\sum_k (\log \Gamma(\beta_0) - \sum_v \log \Gamma(\beta_v) + \sum_v (\beta_v - 1) \log \phi_{v|k}) + 
\sum_d \sum_i \sum_k z_{d_i, k} \log \theta_{k|d} + 
\sum_d \sum_i \sum_k \sum_v w_{d_i, v} z_{d_i, k} \phi_{v|k}$

now that we have the log likelihood, we can fit this to the mean field approach (assuming selective independence/factorization)

$q(\{\theta_d\}, \{\phi_k\}, \{z_{d_i}\})$  
$= (\prod_d q(\phi_d | \lambda_d)) (\prod_k q(\phi_k | \mu_k)) (\prod_d \prod_i q(z_{d_i} | t_{d_i}))$

let $q(z_{d_i}) = \prod_k t_{d_i, k}^{z_{d_i}, k}$ where $\sum t_{d_i, k} = 1$

$ELBO =  E_{q(\theta, \phi, z)}[\log \frac{p(\theta, \phi, z, w)}{q(\theta, \phi, z)}]$

* approach 1: evaluate ELBO and optimize for $q(\cdot)$
* approach 2: use generic solution  
$h_i$ is one hidden variable  
$g(h_i) = E_{q(\text{all except } h_i)}[\log L]$  
$q(h_i) \propto e^{g(h_i)}$

we can note $E_{q(\cdot)} [z_{d_i, k}] = t_{d_i, k}$

E-step for $q(\theta_d | \lambda_d)$

* $g(\theta_d) = E_{q(-\theta_d)}[\sum_k (\alpha_k - 1) \log \theta_{k|d} + \sum_i \sum_k z_{d_i, k} \log \theta_{k|d}]$  
$= \sum_k \log \theta_{k|d} (\alpha_k + \sum_i t_{d_i, k} - 1)$  
then $q(\theta_d) \propto \prod_k \theta_{k|d}^{\alpha_k \sum_i t_{d_i, k} - 1}$  
$\implies q(\theta_d) = Dirichlet(\theta_d | \lambda_{k|d})$  
$\lambda_{k|d} = \alpha_k \sum_i t_{d_i, k}$

E-step for $\phi_k$

* $g(\phi_k) = E_{q(-\phi_k)}[\sum_v (\beta_v - 1) \log \phi_{v|k} + \sum_d \sum_i \sum_v z_{d_i, k} w_{d_i, v} \log \phi_{v|k}$  
$= \sum_v \log \phi_{v|k} (\beta_v + \sum_d \sum_i t_{d_i, k} w_{d_i, v} - 1)$  
$\implies q(\phi_k) = Dirichlet(\phi_k | \mu_{v|k})$  
where $\mu_{v|k} = \beta_v + \sum_d \sum_i t_{d_i, k} w_{d_i, v}$

E-step for $z_d$

* first consider $E[\log \theta_k]$ for $\theta \sim Dirichlet(\alpha)$
* note that for exponential family $p(x | \eta) = h(x) g(\eta) e^{\eta^\top v(x)}$ and $E[v(x)] = -\partial_\eta \log g(\eta)$
* then $\eta_k = \alpha_k - 1$ and $v_k(x) = \log \theta_k$
* then $E[\log \theta_k] = -\partial_{\alpha_k} (\log \Gamma(\sum_k \alpha_k) - \sum_k \log \Gamma(\alpha_k))$ 
$= \psi(\alpha_k) - \psi(\alpha_0)$
* $\psi(x) = \partial_x \log \Gamma(x)$
* getting back to the E-step for $z$ ...  
$g(z_{d_i}) = E_{q(-z_{d_i})}[\sum_k z_{d_i, k} \log \theta_{k|d} + \sum_k \sum_v z_{d_i, k} w_{d_i, k} \log \phi_{v|k}]$  
$= \sum_k z_{d_i, k} (\psi(\lambda_{k|d}) - \psi(\lambda_{0|d}) + \sum_v w_{d_i, v}(\psi(\mu_{v|k}) - \psi(\mu_{0|k})))$
* then $q(z_d{i}) \propto \prod_k (e^{(\cdots)})^{z_{d_i, k}}$
* then $t_{d_i, k} \propto e^{(\cdots)}$
* where $(\cdots) = \psi(\lambda_{k|d}) - \psi(\lambda_{0|d}) + \psi(\mu_{d_i|k}) - \psi(\mu_{0|k})$

M-step

* 